In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datapackage
data_url = 'https://datahub.io/JohnSnowLabs/education-actual-operating-cost-per-student/datapackage.json'
package = datapackage.Package(data_url)
resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])
clean_data = data.dropna()
clean_data.shape

(2190, 10)

In [2]:
school_perf_report = pd.read_csv("oregon_schoool_performance_report_filtered.csv", encoding='iso-8859-1')

In [3]:
school_dist = school_perf_report.groupby(["DISTINSTID","year"])["STUDENRCNT",'FREEREDSTUDCNT','MnrtyStudCnt'].sum()
school_dist.head()

STUDENRCNT  FREEREDSTUDCNT  MnrtyStudCnt
DISTINSTID year                                          
1894       1999      2317.0           865.0         123.0
           2000      2343.0           831.0         186.0
           2001      2199.0           839.0         146.0
           2002      2151.0           875.0         165.0
           2003      2115.0           963.0         176.0

Cross-checking that we get the same school districts for students \>30k and \<100 in dataset from oregon.gov report versus the one from datahub.io. These ids match the ones in [oregon_schools_eda1](oregon_schools_eda1.ipynb), so we can be reasonably confident that our data is good.

In [4]:
school_dist[school_dist["STUDENRCNT"]>30000].index.get_level_values(0).unique()

Int64Index([2142, 2180, 2243], dtype='int64', name='DISTINSTID')

In [5]:
school_dist[school_dist["STUDENRCNT"]<100].index.get_level_values(0).unique()

Int64Index([1895, 1896, 1979, 2010, 2011, 2012, 2015, 2016, 2017, 2018, 2019,
            2020, 2021, 2022, 2023, 2047, 2051, 2052, 2060, 2062, 2063, 2107,
            2109, 2111, 2114, 2115, 2210, 2222, 2247, 2248, 2249],
           dtype='int64', name='DISTINSTID')

Cross comcomparing that the operating cost per student is the same in both reports for the most expensive schools. The Double O elementary school had only one student in 2005 and spent about ~120k. From [oregon_schools_eda1](oregon_schools_eda1.ipynb)

District_ID 	County_Name 	Operating_Cost_Per_Student 	District_Number 	School_Year 	Student_Count 	

2021 	HARNEY 	120444.65 	Double O SD 28 	2005-07-01 	1.0 	

However, school_pref_report dataframe does not have Total Operating Cost or Operating cost per student. But it has about 30 columns which indicate the money spent on the school under various headers. There is no documentation as to what the headers mean, so had to infer their meaning and had to decide if a particular cell had to be included in the operating cost or not, so there might be some error. I tried to identify the columns which are the same across the district/state by grouping and looking for unique values. Calculating the approximate total operating cost from these.

These almost match the "Operating_Cost_Per_Student" schools in [oregon_schools_eda1](oregon_schools_eda1.ipynb).

In [6]:
school_perf_report.columns

Index(['DISTINSTID', 'SCHLNM', 'DISTNM', 'STUDENRCNT', 'DISTSTUDENRCNT',
       'STSTUDENRCNT', 'FREEREDSTUDCNT', 'FREEREDPCT', 'DISTFREEREDPCT',
       'STFREEREDPCT', 'MnrtyStudCnt', 'MnrtyStudPct', 'DistMnrtyStudPct',
       'StMnrtyStudPct', 'GenFundDIRCLSRMAMT', 'GenFundDISTDIRCLSRmAmt',
       'GenFundSTDIRCLSRMAmt', 'GenFundCLSRMSUPPAMt',
       'GenFundDISTCLSRMSUppAmt', 'GenFundSTCLSRMSUPPAmt',
       'GenFundBLDGSUPPAMT', 'GenFundDISTBLDGSUPpAmt', 'GenFundSTBLDGSUPPAmt',
       'GenFundCNTLSUPPAMT', 'GenFundDISTCNTLSUPpAmt', 'GenFundSTCNTLSUPPAmt',
       'GenFundTtlAmt', 'GenFundDistTtlAmt', 'GenFundStTtlAmt',
       'TtlDirClsRmAmt', 'TtlClsRmSuppAmt', 'TtlBldgSuppAmt', 'TtlCntlSuppAmt',
       'TtlSpendAmt', 'TtlDistDirClsRmAmt', 'TtlDistClsRmSuppAmt',
       'TtlDistBldgSuppAmt', 'TtlDistCntlSuppAmt', 'TtlDistSpendAmt',
       'TtlStDirClsRmAmt', 'TtlStClsRmSuppAmt', 'TtlStBldgSuppAmt',
       'TtlStCntlSuppAmt', 'TtlStSpendAmt', 'year'],
      dtype='object')

In [7]:
school_perf_report[(school_perf_report["DISTINSTID"]==2021) & (school_perf_report["year"]==2005)].transpose()

,4781
DISTINSTID,2021
SCHLNM,double o elem
DISTNM,Double O SD 28
STUDENRCNT,1
DISTSTUDENRCNT,1
STSTUDENRCNT,556170
FREEREDSTUDCNT,1
FREEREDPCT,100
DISTFREEREDPCT,100
STFREEREDPCT,42.18


In [8]:
# Example grouping and looking for unique values to isolate the 
# school costs.
school_perf_report.groupby(["DISTINSTID","year"])["TtlSpendAmt"].unique()

DISTINSTID  year
1894        1999    [7730.28, 6509.39, nan, 22118.25, 6624.98, 666...
            2000    [7847.66, 6611.38, nan, 22133.2, 7745.09, 6572...
            2001    [7706.0, 6928.98, 6935.13, 6980.81, 8234.45, 6...
            2002    [7404.02, 6482.85, 8874.36, 9679.8, 6585.39, 6...
            2003    [7861.59, 7159.21, 7968.21, 9849.19, 6985.83, ...
            2004    [8238.59, 7182.31, 7551.96, 9451.37, 7548.14, ...
            2005    [7303.03, 7024.88, 8034.09, 8046.89, 7341.9, 7...
            2006    [8737.17, 8271.35, 9600.91, 12477.45, 8741.35,...
            2007    [9594.75, 8991.42, 11347.39, 11310.55, 9474.68...
            2008    [9995.94, 9439.6, 10499.68, 11403.16, 9917.47,...
            2009    [10026.15, 9049.09, 10804.86, 10061.23, 9083.2...
            2010    [9121.35, 8325.13, 9778.74, 10772.91, 8986.52,...
            2011    [9054.22, 8093.87, 10757.26, 10678.64, 8785.8,...
            2012    [9949.74, 8828.41, 10861.51, 9989.18, 8503.12,...
   

In [9]:
import re
pat = re.compile(r"AMT",re.IGNORECASE)
def total_spending(row):
    sum = 0
    for idx in row.index:
        if re.search(r"AMT",idx,re.IGNORECASE) and not np.isnan(row[idx]):
            if re.search(r"GenFundTtlAmt",idx, re.IGNORECASE) or \
                re.search(r"TtlDirClsRmAmt",idx, re.IGNORECASE) or \
                re.search(r"TtlClsRmSuppAmt", idx, re.IGNORECASE) or \
                re.search(r"TtlBldgSuppAmt", idx, re.IGNORECASE) or \
                re.search(r"TtlCntlSuppAmt", idx):
                sum = sum + row[idx]
                pass
            elif not re.search(r"TTL", idx,re.IGNORECASE):
                sum = sum + row[idx]
    row["Total Spending"] = sum
    row["Operating Cost Per Student"] = sum/row["STUDENRCNT"]
    return row
    
school_perf_report = school_perf_report.apply(total_spending, axis=1)

In [10]:
school_perf_report[(school_perf_report["DISTINSTID"]==2021) & (school_perf_report["year"]==2005)]["Total Spending"]


4781    112940.77758
Name: Total Spending, dtype: float64

In [11]:
school_perf_report[(school_perf_report["Operating Cost Per Student"]>50000) & (school_perf_report["year"]<2010) ][["DISTINSTID","SCHLNM","DISTNM","STUDENRCNT","Operating Cost Per Student","year"]]

,DISTINSTID,SCHLNM,DISTNM,STUDENRCNT,Operating Cost Per Student,year
3639,1979,brors elem,BROTHERS SCH DIST 15,3.0,86229.863260,2003
3640,1979,brors elem,BROTHERS SCH DIST 15,3.0,75841.121921,2004
4781,2021,double o elem,Double O SD 28,1.0,112940.777580,2005
4783,2021,double o elem,Double O SD 28,1.0,117951.169639,2007
6005,2051,ashwood elem,Ashwood SD 8,1.0,178176.103160,2008
16483,2222,troy elem,Troy SD 54,2.0,181221.176633,2006
16484,2222,troy elem,Troy SD 54,2.0,185046.919819,2007
22553,2241,ellen stevens community academy,Forest Grove SD 15,5.0,57185.822923,2007
22908,2107,rockville elem,Jordan Valley SD 3,2.0,58775.574972,2004
